# Project - 1 : Apache Spark 

Instructor: Ramesh Yerraballi
TA: Madhumitha Sakthi
Semester: Fall 2019
Due Date: 11:59pm, Monday 9/16

This project is based on Map-Reduce Framework. In these you will get to work with Spark and will get to know how 
does spark work, what functionalities does spark provide, what does map-reduce framework do and why is it useful. 

In this project you will be implementing a basic song recommender system. You will be given a dataset where there are multiple csv files. These csv files have data corresponding to song play count and song information.

The data you would be using will be provided in a zip file along with this notebook. The __msd.zip__ archive contains '_kaggle_visible_evaluation_triplets.txt_'. We will be using the visible part of the testing data to understand the working on Apache Spark.  The user's listening history is provided as: (user, song, play count). In another file, '_kaggle_songs.txt_', each song is marked using an index for easier representation of songs. 



What to turn in:
A zip folder which will have:
- Jupyter Notebook
- A brief report on what features you used for recommendation. And a brief explanation of flow of your code. For example,  what RDD does what or, why it was created.
- datasets folder with the csv files you are using in your notebook.
- Notebook should use relative path to the csv files in datasets folder.
- Name of the zip folder - <your\_name>\_<your\_partner_name>.zip


This project consists of 4 questions. 
1. Create an RDD with _msd_evalutation_triplets.txt_ and replace the song name with the song index from _msd_songs.txt_. Identify the number of songs that do not have any rating. 
2. Generate song ratings based on the song play count as a normalized score between 0 and 1. 
3. Identify the popular song based on this rating and recommend songs to user, given user id based on the algorithm used in Movie recommender system from class. 
4. Using Cosine similarity function, identify pair-wise similarity between each pair of users and generate the top 5 most similar users without an overlap in users. 

The above list is the higer level idea about the questions. 

In [1]:
### Starter code ####

##### These lines are to tell jupyter where to find Apache Spark ####
import findspark
# findspark.init('C:\\apachespark')
from pyspark import SparkConf, SparkContext
from itertools import accumulate
conf = SparkConf().setMaster("local[*]").setAppName("Songs")
# There are two configurable parameters
# 1. A cluster URL, namely  local  in this example, which tells Spark how to connect
# to a cluster.  local  is a special value that runs Spark on one thread on the local
# machine, without connecting to a cluster.
# 2. An application name, namely  Movies  in this example. This will identify your
# application on the cluster manager’s UI if you connect to a cluster.
sc = SparkContext(conf = conf)
##### These lines are to tell jupyter where to find Apache Spark ####

In [2]:
## Read triplet file into RDD
triplet_rdd = sc.textFile(r"msd/kaggle_visible_evaluation_triplets.txt") \
    .map(lambda line: line.split("\t")) 

## Read song file into RDD
songs_rdd = sc.textFile(r"msd/kaggle_songs.txt") \
    .map(lambda line: line.split(" ")) 

## Step 1: 
Replace song name with song index and identify the number of songs without user history

In [3]:
hacked_triplets = triplet_rdd.map(lambda x: (1,x))

In [4]:
hacked_songs = songs_rdd.map(lambda x: (1,x))

In [5]:
hacked_triplets.take(20)

[(1, ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', 'SOBONKR12A58A7A7E0', '1']),
 (1, ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', 'SOEGIYH12A6D4FC0E3', '1']),
 (1, ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', 'SOFLJQZ12A6D4FADA6', '1']),
 (1, ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', 'SOHTKMO12AB01843B0', '1']),
 (1, ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', 'SODQZCY12A6D4F9D11', '1']),
 (1, ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', 'SOXLOQG12AF72A2D55', '1']),
 (1, ['d7083f5e1d50c264277d624340edaaf3dc16095b', 'SOUVUHC12A67020E3B', '1']),
 (1, ['d7083f5e1d50c264277d624340edaaf3dc16095b', 'SOUQERE12A58A75633', '1']),
 (1, ['d7083f5e1d50c264277d624340edaaf3dc16095b', 'SOIPJAX12A8C141A2D', '1']),
 (1, ['d7083f5e1d50c264277d624340edaaf3dc16095b', 'SOEFCDJ12AB0185FA0', '2']),
 (1, ['d7083f5e1d50c264277d624340edaaf3dc16095b', 'SOATCSU12A8C13393A', '1']),
 (1, ['d7083f5e1d50c264277d624340edaaf3dc16095b', 'SOZPZGN12A8C135B45', '1']),
 (1, ['d7083f5e1d50c264277d624340edaaf3dc16095b', 'S

In [6]:
songs_rdd.take(10)

[['SOAAADD12AB018A9DD', '1'],
 ['SOAAADE12A6D4F80CC', '2'],
 ['SOAAADF12A8C13DF62', '3'],
 ['SOAAADZ12A8C1334FB', '4'],
 ['SOAAAFI12A6D4F9C66', '5'],
 ['SOAAAGK12AB0189572', '6'],
 ['SOAAAGN12AB017D672', '7'],
 ['SOAAAGO12A67AE0A0E', '8'],
 ['SOAAAGP12A6D4F7D1C', '9'],
 ['SOAAAGQ12A8C1420C8', '10']]

In [7]:
def matchSongName(row):
    if row[1] == row[2]:
        return True
    return False

In [8]:
# RDD With Song Index instead of Song Name
joined = hacked_triplets.join(hacked_songs) \
            .map(lambda x: [x[1][0][0] , x[1][0][1] , x[1][1][0] , x[1][1][1] , x[1][0][2]]) \
            .filter(matchSongName) \
            .map(lambda x: [x[0] , x[3] , x[4]])

joined.take(10)

[['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', '25150', '1'],
 ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', '68212', '1'],
 ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', '87433', '1'],
 ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', '123630', '1'],
 ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', '58821', '1'],
 ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', '351764', '1'],
 ['d7083f5e1d50c264277d624340edaaf3dc16095b', '315822', '1'],
 ['d7083f5e1d50c264277d624340edaaf3dc16095b', '312787', '1'],
 ['d7083f5e1d50c264277d624340edaaf3dc16095b', '136848', '1'],
 ['d7083f5e1d50c264277d624340edaaf3dc16095b', '67457', '2']]

In [9]:
onlySongNames = triplet_rdd.map(lambda x: x[1]).collect()

In [10]:
song_set = set(onlySongNames)

In [11]:
# Number of missing songs
distinctSongs = songs_rdd.keys().collect()
missing_songs = len(distinctSongs) - len(song_set)
print(str(missing_songs) + " songs do not have a user history.")

223007 songs do not have a user history.


## Step 2:
Generate song ratings based on the play_count. For example, if (song_1, 5; song_2, 10; song_3, 5) i.e., song_1 is played 5 times, song_2 is played 10 times and song_3 is played 5 times, the normalized rating score should be 0.25, 0.5 and 0.25 respectively. 
Similarly, generate the rating for all the songs. You may notice that based on all songs, the rating is almost always very low. So, think of the best way to convert song count to ratings. (Hint: Try generating ratings based on each user's song play history)

In [27]:
song_user_grouped = triplet_rdd.map(lambda x: [x[0], [x[1], int(x[2])]])

In [49]:
song_user_sum = triplet_rdd.map(lambda x: (x[0], int(x[2]))) \
                        .reduceByKey(lambda x,y: x+y)


In [50]:
song_user_sum.take(10)

[('d7083f5e1d50c264277d624340edaaf3dc16095b', 17),
 ('d68dc6fc25248234590d7668a11e3335534ae4b4', 6),
 ('fdf6afb5daefb42774617cf223475c6013969724', 10),
 ('10cbcd627472477dfbec90fb75017f8df6ce84ec', 9),
 ('30e4a688e6fc9c8bfe55998af3996a909ae34449', 8),
 ('da7bc0ec91a21a54f0b209bcc9ec5b4b49613a68', 129),
 ('3a613180775197cd08c154abe4e3f67af238a632', 50),
 ('4b9b549e84fb29bfbcf7ab34f01c0bcd1bbf93a5', 97),
 ('5b7124d06fe4b027b6dff487da9ac236aa4fc3e4', 31),
 ('6530c4fc41b9110de5d39fe0355fa103c66385f0', 44)]

In [52]:
user_ratings = triplet_rdd.join(song_user_sum)
user_ratings.take(10)

[('d7083f5e1d50c264277d624340edaaf3dc16095b', ('SOUVUHC12A67020E3B', 17)),
 ('d7083f5e1d50c264277d624340edaaf3dc16095b', ('SOUQERE12A58A75633', 17)),
 ('d7083f5e1d50c264277d624340edaaf3dc16095b', ('SOIPJAX12A8C141A2D', 17)),
 ('d7083f5e1d50c264277d624340edaaf3dc16095b', ('SOEFCDJ12AB0185FA0', 17)),
 ('d7083f5e1d50c264277d624340edaaf3dc16095b', ('SOATCSU12A8C13393A', 17)),
 ('d7083f5e1d50c264277d624340edaaf3dc16095b', ('SOZPZGN12A8C135B45', 17)),
 ('d7083f5e1d50c264277d624340edaaf3dc16095b', ('SOPFVWP12A6D4FC636', 17)),
 ('d7083f5e1d50c264277d624340edaaf3dc16095b', ('SOHEKND12A8AE481D0', 17)),
 ('d7083f5e1d50c264277d624340edaaf3dc16095b', ('SOPSVVG12A8C13B444', 17)),
 ('d7083f5e1d50c264277d624340edaaf3dc16095b', ('SODSKZZ12AB0188524', 17))]

In [16]:
# Each Song with it's play count summed across the dataset
song_user_rated_per_user = song_user_grouped.reduceByKey(lambda x,y:x+y)

In [17]:
song_user_rated_per_user.take(10)

[('d7083f5e1d50c264277d624340edaaf3dc16095b',
  [['SOUVUHC12A67020E3B', 1],
   ['SOUQERE12A58A75633', 1],
   ['SOIPJAX12A8C141A2D', 1],
   ['SOEFCDJ12AB0185FA0', 2],
   ['SOATCSU12A8C13393A', 1],
   ['SOZPZGN12A8C135B45', 1],
   ['SOPFVWP12A6D4FC636', 1],
   ['SOHEKND12A8AE481D0', 1],
   ['SOPSVVG12A8C13B444', 1],
   ['SODSKZZ12AB0188524', 1],
   ['SONZTNP12A8C1321DF', 1],
   ['SOVVLKF12A8C1424F0', 1],
   ['SOMLKZO12AB017F4AE', 1],
   ['SOACRJG12A8C137A8D', 1],
   ['SONJVYU12A8AE44F9E', 1],
   ['SOSOUKN12A8C13AB79', 1]]),
 ('d68dc6fc25248234590d7668a11e3335534ae4b4',
  [['SOFRQTD12A81C233C0', 1],
   ['SOZQIUZ12A8C13CFBE', 1],
   ['SOKQNYH12A6D4FA5D3', 1],
   ['SOQDMED12A67ADE731', 1],
   ['SOAXGDH12A8C13F8A1', 1],
   ['SOAAGFH12A8C13D072', 1]]),
 ('fdf6afb5daefb42774617cf223475c6013969724',
  [['SOJZBHH12AB017F611', 1],
   ['SOONUTJ12A6701D7B4', 1],
   ['SONXYJW12AB018898A', 1],
   ['SONDFYA12AB018897F', 1],
   ['SOATBYQ12AB0188962', 1],
   ['SOWDRCH12A8159E8B4', 1],
   ['SOJNPGW12AB01

In [18]:
songs_count_unique.take(10)

[('SOBONKR12A58A7A7E0', 35432),
 ('SOFLJQZ12A6D4FADA6', 7895),
 ('SOHTKMO12AB01843B0', 10515),
 ('SOXLOQG12AF72A2D55', 4671),
 ('SOZPZGN12A8C135B45', 39),
 ('SOPFVWP12A6D4FC636', 149),
 ('SODSKZZ12AB0188524', 27),
 ('SOACRJG12A8C137A8D', 12),
 ('SOSOUKN12A8C13AB79', 134),
 ('SOFRQTD12A81C233C0', 19454)]

In [19]:
sorted_songs = songs_count_unique.sortBy(lambda x: -x[1])
sorted_songs.take(10)

[('SOBONKR12A58A7A7E0', 35432),
 ('SOAUWYT12A81C206F1', 33179),
 ('SOSXLTC12AF72A7F54', 24359),
 ('SOFRQTD12A81C233C0', 19454),
 ('SOEGIYH12A6D4FC0E3', 17115),
 ('SOAXGDH12A8C13F8A1', 14279),
 ('SONYKOW12AB01849C9', 12392),
 ('SOVDSJC12A58A7A271', 11610),
 ('SOUFTBI12AB0183F65', 10794),
 ('SOHTKMO12AB01843B0', 10515)]

In [20]:
def rateSong(row):
    rating = round((10/35432)*row[1],2)
    rate = [row[0], rating]
    return rate

In [22]:
rated_songs = songs_count_unique.map(rateSong)

In [81]:
# Songs rated on the number of times it was played by any user - Highest played song=10, Songs played once is lowest~0
rated_songs.take(10)
#Top ten songs with ratings

[['SOBONKR12A58A7A7E0', 10.0],
 ['SOFLJQZ12A6D4FADA6', 2.23],
 ['SOHTKMO12AB01843B0', 2.97],
 ['SOXLOQG12AF72A2D55', 1.32],
 ['SOZPZGN12A8C135B45', 0.01],
 ['SOPFVWP12A6D4FC636', 0.04],
 ['SODSKZZ12AB0188524', 0.01],
 ['SOACRJG12A8C137A8D', 0.0],
 ['SOSOUKN12A8C13AB79', 0.04],
 ['SOFRQTD12A81C233C0', 5.49]]

In [96]:
hacked_songkey_triplets = hacked_triplets.map(lambda x: (x[1][1], [x[1][0], x[1][2]]))

In [97]:
hacked_songkey_triplets.take(10)

[('SOBONKR12A58A7A7E0', ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', '1']),
 ('SOEGIYH12A6D4FC0E3', ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', '1']),
 ('SOFLJQZ12A6D4FADA6', ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', '1']),
 ('SOHTKMO12AB01843B0', ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', '1']),
 ('SODQZCY12A6D4F9D11', ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', '1']),
 ('SOXLOQG12AF72A2D55', ['fd50c4007b68a3737fe052d5a4f78ce8aa117f3d', '1']),
 ('SOUVUHC12A67020E3B', ['d7083f5e1d50c264277d624340edaaf3dc16095b', '1']),
 ('SOUQERE12A58A75633', ['d7083f5e1d50c264277d624340edaaf3dc16095b', '1']),
 ('SOIPJAX12A8C141A2D', ['d7083f5e1d50c264277d624340edaaf3dc16095b', '1']),
 ('SOEFCDJ12AB0185FA0', ['d7083f5e1d50c264277d624340edaaf3dc16095b', '2'])]

## Step 3: 
For a given user_id, rating, recommend 5 other songs from the list. One way to do this is based on another user who liked the same song liked by this user with rating more than the given rating and recommend the 5 songs based on the matched user's rating. 

In [75]:
def songFilter(row):
    if (x[0]==user_id and ):
        

SyntaxError: unexpected EOF while parsing (<ipython-input-75-2e17c8beb54b>, line 2)

In [114]:
combined_rated_triplets = rated_songs.join(hacked_songkey_triplets) \
                                    .map(lambda x: [(x[0],x[1][0]), [x[1][1][0], x[1][1][1]] ]) \
                                    .groupByKey() \
                                    .map(lambda x : (x[0], list(x[1])))
#                                     .sortedBy(x[0][1])

In [116]:
combined_rated_triplets.take(5)

[(('SOACRJG12A8C137A8D', 0.0),
  [['d7083f5e1d50c264277d624340edaaf3dc16095b', '1'],
   ['5c77aaafc7ecabdb45fda3b904fa695eeb98b217', '6'],
   ['9672fcd7fbf2222a470e8c3055e716e332024f99', '1'],
   ['3648ee52846b074dfafec3fbafd068c43dabbe13', '1'],
   ['396bf84cf84fb253b17e6eecd43de3c71f27b178', '2'],
   ['853467ed34af34df745d7e9311955b8cc364b38e', '1']]),
 (('SOFIPHI12AAF3B3DB2', 0.12),
  [['9be82340a8b5ef32357fe5af957ccd54736ece95', '1'],
   ['75f21ebb6e7cbc2816197d80dfa66c7a8a069b5f', '18'],
   ['db0bff23e21e1076ebb381201644892e7facbce5', '1'],
   ['305ead5b95ca66fea458fb09552497179f97f42b', '2'],
   ['672cddfdc7b7cd37cf929afc578692657dda3c75', '1'],
   ['1fbe69ad8fa984ba383b164d8562b9d2adfc5576', '1'],
   ['a9a2eabde57a5935c2034005593bb7f6ec2067fa', '1'],
   ['c6de408082517acef878dd1fa9e4794dc72b5684', '7'],
   ['e9ef744999202ef2fe6e7bf632ca03c9d49b0774', '2'],
   ['2c9386c51a582623f77a9b5fdd617740ba4744a6', '1'],
   ['898162ea7b2336e6a56e10813d2fcc70008c7942', '1'],
   ['fe6854ae9b0

In [118]:
user_id=input("Enter user ID: ")
minrating=input("Enter minimum rating: ")



Enter user ID: 864c7d3f68bc4590114ebc54b7a1c3a9afce4084
Enter minimum rating: 6


In [37]:
givenuser_songs = triplet_rdd.filter(songFilter)

In [41]:
u_songnames = givenuser_songs.map(lambda x: x[1])

In [ ]:
def findUsersFromSong(row):
    if row[1]==

In [40]:
songs_other_users = triplet_rdd.filter(findUsersFromSong)

Traceback (most recent call last):
  File "/Users/subhayuchakravarty/anaconda3/lib/python3.7/site-packages/pyspark/serializers.py", line 590, in dumps
    return cloudpickle.dumps(obj, 2)
  File "/Users/subhayuchakravarty/anaconda3/lib/python3.7/site-packages/pyspark/cloudpickle.py", line 863, in dumps
    cp.dump(obj)
  File "/Users/subhayuchakravarty/anaconda3/lib/python3.7/site-packages/pyspark/cloudpickle.py", line 260, in dump
    return Pickler.dump(self, obj)
  File "/Users/subhayuchakravarty/anaconda3/lib/python3.7/pickle.py", line 437, in dump
    self.save(obj)
  File "/Users/subhayuchakravarty/anaconda3/lib/python3.7/pickle.py", line 504, in save
    f(self, obj) # Call unbound method with explicit self
  File "/Users/subhayuchakravarty/anaconda3/lib/python3.7/pickle.py", line 786, in save_tuple
    save(element)
  File "/Users/subhayuchakravarty/anaconda3/lib/python3.7/pickle.py", line 504, in save
    f(self, obj) # Call unbound method with explicit self
  File "/Users/sub

PicklingError: Could not serialize object: Exception: It appears that you are attempting to broadcast an RDD or reference an RDD from an action or transformation. RDD transformations and actions can only be invoked by the driver, not inside of other transformations; for example, rdd1.map(lambda x: rdd2.values.count() * x) is invalid because the values transformation and count action cannot be performed inside of the rdd1.map transformation. For more information, see SPARK-5063.

## Step 4: 
1. Compute cosine similarity between all pairs of users. 
2. Sort the similarity score and print the top-5 similar users. 
3. If the top-5 user set has an user appearing more than once, ignore that pair and take the next best pair from the sorted list. 
4. For a given user_id, identify the top-5 similar users and hence song recommendations from other user's list. 

In [65]:
numbers = sc.parallelize([[1,7],[8,9],[1,7],[8,8],[1,3],[1,5]])
numbers.collect()

[[1, 7], [8, 9], [1, 7], [8, 8], [1, 3], [1, 5]]

In [56]:
x = numbers.map(lambda x: x[1]).sum()
x


31

In [58]:
from itertools import accumulate

In [69]:
sumx = numbers.groupByKey().mapValues(cumsum)
# numbers.groupByKey().mapValues(cumsum).flatMapValues(lambda x:x).collect()

summ = sumx.map(lambda x: [x[0],x[1][-1]])
summ.collect()

[[8, 17], [1, 22]]